Dashboard

In [2]:
from influxdb import DataFrameClient
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
import pandas as pd

In [57]:
bucket = "DSI_test"
token ="GMbnHWGhM9p9t9mIjbc1I5KlWir8LJxBDkpMF0SiOa56f1nvLepCEN7iI_5-sR80FA8CvLmf_mHcy8Gc5XYwvA=="
org="dsi"
client = InfluxDBClient(url="http://localhost:8086", token=token, org=org)

query_api = client.query_api()

symbol ="CRM"
display_range = "5d"

query = """from(bucket: "DSI_test")
        |> range(start: -"""+display_range+""")
        |> filter(fn: (r) => r["_measurement"] ==\"""" +symbol+"""\")
        |> filter(fn: (r) => r["_field"] == "Open" or r["_field"] == "Close" or r["_field"] == "High" or r["_field"] == "Low")"""

result = query_api.query(org=org, query=query)

In [51]:
query

'from(bucket: "DSI_test")\n        |> range(start: -5d)\n        |> filter(fn: (r) => r["_measurement"] =="CRM")\n        |> filter(fn: (r) => r["_field"] == "Open" or r["_field"] == "Close" or r["_field"] == "High" or r["_field"] == "Low")'

In [55]:
results = []
for table in result:
  for record in table.records:
    results.append((record.get_time(), record.get_measurement(), record.get_field(), record.get_value()))


results

[(datetime.datetime(2023, 1, 20, 14, 30, tzinfo=datetime.timezone.utc),
  'CRM',
  'Close',
  145.7700042725),
 (datetime.datetime(2023, 1, 20, 14, 31, tzinfo=datetime.timezone.utc),
  'CRM',
  'Close',
  145.6000061035),
 (datetime.datetime(2023, 1, 20, 14, 32, tzinfo=datetime.timezone.utc),
  'CRM',
  'Close',
  145.6150054932),
 (datetime.datetime(2023, 1, 20, 14, 33, tzinfo=datetime.timezone.utc),
  'CRM',
  'Close',
  145.3000030518),
 (datetime.datetime(2023, 1, 20, 14, 34, tzinfo=datetime.timezone.utc),
  'CRM',
  'Close',
  145.3099975586),
 (datetime.datetime(2023, 1, 20, 14, 35, tzinfo=datetime.timezone.utc),
  'CRM',
  'Close',
  145.125),
 (datetime.datetime(2023, 1, 20, 14, 36, tzinfo=datetime.timezone.utc),
  'CRM',
  'Close',
  145.6999969482),
 (datetime.datetime(2023, 1, 20, 14, 37, tzinfo=datetime.timezone.utc),
  'CRM',
  'Close',
  145.7599945068),
 (datetime.datetime(2023, 1, 20, 14, 38, tzinfo=datetime.timezone.utc),
  'CRM',
  'Close',
  145.6699981689),
 (dateti

In [65]:
#make results a dataframe
df = pd.DataFrame(results, columns=['Date',"Symbol", 'OHLC', "Value"])
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Symbol,OHLC,Value
0,2023-01-20 14:30:00+00:00,CRM,Close,145.770004
1,2023-01-20 14:31:00+00:00,CRM,Close,145.600006
2,2023-01-20 14:32:00+00:00,CRM,Close,145.615005
3,2023-01-20 14:33:00+00:00,CRM,Close,145.300003
4,2023-01-20 14:34:00+00:00,CRM,Close,145.309998


In [66]:
#if OHLC is Open, High, Low, or Close, then make it a column
df = df.pivot(index='Date', columns='OHLC', values='Value')
df.head()

OHLC,Close,High,Low,Open
Date,,,,
2023-01-20 14:30:00+00:00,145.770004,146.000000,144.300003,144.300003
2023-01-20 14:31:00+00:00,145.600006,145.785004,145.119995,145.770004
2023-01-20 14:32:00+00:00,145.615005,146.199997,145.289993,145.529999
2023-01-20 14:33:00+00:00,145.300003,145.589996,144.759995,145.589996
2023-01-20 14:34:00+00:00,145.309998,145.609894,145.145004,145.145004


In [71]:
#build plotly stock graph
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.show()